In [120]:
import pandas as pd
import re
import emoji
import nagisa
import neologdn

In [4]:
dataset_path = "../data/"

In [30]:
train_df = pd.read_csv(f"{dataset_path}1.train_data.csv")
test_df = pd.read_csv(f"{dataset_path}2.test_data.csv")
test_df_flg = pd.read_csv(f"{dataset_path}3.submission.csv")
test_df = pd.concat([test_df, test_df_flg], join='inner', axis=1)

In [31]:
train_df.head()

,id,flg,keyword,location,text
0,20587,0,覚醒剤,icon:大トロ(@sato_wall_aim ),【マイリスト】カカカカ覚醒剤 https://t.co/15ZY9Oa2QS #sm3630...
1,31857,0,雷 落雷,ジャングル,@kenjikenji3343 せやな 落雷撃たれたら全員張り付くしかなかったからなぁ なん...
2,36793,0,血液,↓↓↓↓↓こちらをクリック↓↓↓↓,【カフェインの力】 体を動かす前にカフェインを摂ると交感神経が刺激され、体内の脂肪が分解され...
3,42299,0,ハリケーン,Close to your heart,あなたの鼓膜に直に響かせたい ハリケーンのように激しく愛したい 余計な壁なんて取っ払って 奥...
4,51449,0,殺人鬼,偽物なので悪しからず,@Mr8yt 違う違う。お化けは現実にいないけど殺人鬼は現実にいるじゃん〜。うーん、ホラーっ...


In [32]:
test_df.head()

,id,keyword,location,text,flg
0,25429,避難した,日本,@adachib 反原発によるエビデンスのない恐怖や不安煽りで自主避難した人を蹴落とすのかな？,0
1,40004,火災,NaN,通行止【2月2日14:25現在】八戸道 安代ＪＣＴ〜浄法寺ＩＣ (下り北行) 火災による通...,0
2,33451,目撃者,NaN,君は死ななくてはならないんだ…目撃者は生かしておけないよ…誰ひとりとしてこの『吉良吉影』の正...,0
3,44781,負傷者,えいてぃーん / ic → りーふさん,寸止めガチ勢任せてください。(負傷者2人以上の技師本体は無理),0
4,27007,衝突,ハリー先生の元,暇だったのでみんなでごっこ遊びをしていた時 ハリー「おっと、それ以上近づくと軽快なBGMと共...,0


In [42]:
train_df.info()
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13524 entries, 0 to 13523
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        13524 non-null  int64 
 1   keyword   13524 non-null  object
 2   location  7944 non-null   object
 3   text      13524 non-null  object
 4   flg       13524 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 528.4+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20285 entries, 0 to 20284
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        20285 non-null  int64 
 1   flg       20285 non-null  int64 
 2   keyword   20285 non-null  object
 3   location  11819 non-null  object
 4   text      20285 non-null  object
dtypes: int64(2), object(3)
memory usage: 792.5+ KB


In [44]:
train_df['flg'].value_counts()

0    18851
1     1434
Name: flg, dtype: int64

In [45]:
# testデータに正例が1つも含まれていない
test_df['flg'].value_counts()

0    13524
Name: flg, dtype: int64

## リプライ
- リプライの多くは負例にふくまれている
- 災害に関する有益な情報は多くの人に共有されることが影響

In [61]:
train_df['text'].str.startswith('@').value_counts()

False    15675
True      4610
Name: text, dtype: int64

In [64]:
train_df[train_df.flg == 0]['text'].str.startswith('@').sum()

4530

In [66]:
train_df[train_df.flg == 1]['text'].str.startswith('@').sum()

80

In [35]:
test_df[test_df.flg == 0]['text'].str.startswith('@').sum()

2997

In [36]:
test_df[test_df.flg == 1]['text'].str.startswith('@').sum()

0

## URL
- 正例の多くはURL付きツイート（学習データだと 951/1434がURL付きツイート）
- また，URL付きツイートの951/5190が正例

In [50]:
def inURL(text: str):
    return re.search(r'https?://[\w\:%#$&\?\(\)!\.=\+\-]+', text) is not None

In [70]:
train_df['inurl'] = train_df.text.apply(inURL)
test_df['inurl'] = test_df.text.apply(inURL)

In [60]:
train_df.inurl.value_counts()

False    15095
True      5190
Name: inurl, dtype: int64

In [67]:
train_df[train_df.flg == 1].inurl.sum()

951

In [68]:
train_df[train_df.flg == 0].inurl.sum()

4239

In [71]:
test_df.inurl.value_counts()

False    10011
True      3513
Name: inurl, dtype: int64

## location
- locationが書かれているユーザの方が多い
- 別途分析が必要そう

In [74]:
train_df.location.isnull().sum()

8466

In [82]:
print(train_df.flg.value_counts())
train_df[train_df.location.isnull()].flg.value_counts()

0    18851
1     1434
Name: flg, dtype: int64


0    7921
1     545
Name: flg, dtype: int64

In [90]:
train_df[train_df.count(axis=1) == 6].flg.value_counts()

0    10930
1      889
Name: flg, dtype: int64

### locationがNanでないもの

In [89]:
train_df[train_df.count(axis=1) == 6].location

0        icon:大トロ(@sato_wall_aim )
1                            ジャングル
2                ↓↓↓↓↓こちらをクリック↓↓↓↓
3              Close to your heart
4                       偽物なので悪しからず
                   ...            
20277                       仙台市青葉区
20278                         尿道の中
20281                   🐒🐟 ✩͓̊  🌊🐷
20282                        Japan
20284                       日本のどこか
Name: location, Length: 11819, dtype: object

In [91]:
with open(f'{dataset_path}location_list.txt', 'w') as f:
    f.write('\n'.join(train_df[train_df.count(axis=1) == 6].location.tolist()))

## hashtag
- 正例の 380/1434にhashtagがふくまれている

In [93]:
def in_hashtag(text: str):
    return re.search(r'#.+', text) is not None

In [98]:
train_df['inhashtag'] = train_df.text.apply(in_hashtag)
test_df['inhashtag'] = test_df.text.apply(in_hashtag)

In [99]:
train_df.inhashtag.value_counts()

False    18165
True      2120
Name: inhashtag, dtype: int64

In [100]:
train_df[train_df.flg == 1].inhashtag.sum()

380

In [101]:
train_df[train_df.flg == 0].inhashtag.sum()

1740

In [102]:
test_df.inhashtag.value_counts()

False    12052
True      1472
Name: inhashtag, dtype: int64

#### urlとhashtagの両方が含まれる場合，1/4以上の確率で正例

In [107]:
train_df[train_df.inurl][train_df.inhashtag].flg.value_counts()

/Users/rikeda/Development/DisasterPrediction/env/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


0    1055
1     338
Name: flg, dtype: int64

## 絵文字
- 絵文字は真剣なツイート中に出現しにくいため，正例にはほとんど出現しない

In [108]:
def in_emoji(text: str):
    for c in text:
        if c in emoji.UNICODE_EMOJI:
            return True
    return False

In [109]:
train_df['inemoji'] = train_df.text.apply(in_emoji)
test_df['inemoji'] = test_df.text.apply(in_emoji)

In [110]:
train_df.inemoji.value_counts()

False    17849
True      2436
Name: inemoji, dtype: int64

In [111]:
train_df[train_df.flg == 1].inemoji.sum()

60

In [112]:
train_df[train_df.flg == 0].inemoji.sum()

2376

In [113]:
test_df.inemoji.value_counts()

False    11917
True      1607
Name: inemoji, dtype: int64

## 顔文字
- 顔文字も雑なルールで抽出したが，顔文字を含むツイートは負例が多い
- 絵文字も顔文字も真面目な場面では使われにくいことが影響か

In [130]:
def in_kaomoji(text: str):
    m = re.compile(r'\([^あ-ん\u30A1-\u30F4\u2E80-\u2FDF\u3005-\u3007\u3400-\u4DBF\u4E00-\u9FFF\uF900-\uFAFF\U00020000-\U0002EBEF]+?\)')
    return m.search(text) is not None

In [131]:
train_df['inkaomoji'] = train_df.text.apply(in_kaomoji)
test_df['inkaomoji'] = test_df.text.apply(in_kaomoji)

In [132]:
train_df.inkaomoji.value_counts()

False    19150
True      1135
Name: inkaomoji, dtype: int64

In [133]:
train_df[train_df.flg == 1].inkaomoji.sum()

72

In [134]:
train_df[train_df.flg == 0].inkaomoji.sum()

1063

In [135]:
test_df.inkaomoji.value_counts()

False    12695
True       829
Name: inkaomoji, dtype: int64

## keywords

In [138]:
train_df.keyword.value_counts()

抹消する             264
雪崩               251
燃えている            245
警察               242
検査               241
                ... 
暴力 爆撃              1
衝突 大破 正面衝突         1
目撃者 目撃証言 殺人者       1
虐殺 窃盗 殺人者 殺人鬼      1
衝突 負傷 正面衝突         1
Name: keyword, Length: 315, dtype: int64

## 数字
- 正例のほとんど(1183/1434)に数字系列がふくまれているので有効な素性の可能性がある

In [139]:
def in_number(text: str):
    return re.search(r'\d+', text) is not None

In [140]:
train_df['innum'] = train_df.text.apply(in_number)
test_df['innum'] = test_df.text.apply(in_number)

In [141]:
train_df.innum.value_counts()

False    10623
True      9662
Name: innum, dtype: int64

In [142]:
train_df[train_df.flg == 1].innum.sum()

1183

In [143]:
train_df[train_df.flg == 0].innum.sum()

8479

In [144]:
test_df.innum.value_counts()

False    7089
True     6435
Name: innum, dtype: int64